### Documnets clustering using Opinion Review Data set

In [1]:
import pandas as pd
import glob, os

In [6]:
path = r'D:\workspace1\github\1_python_ml_guide\UCI_opinion_review'

In [2]:
glob.glob?

Signature: glob.glob(pathname, *, recursive=False)
Docstring:
Return a list of paths matching a pathname pattern.

The pattern may contain simple shell-style wildcards a la
fnmatch. However, unlike fnmatch, filenames starting with a
dot are special cases that are not matched by '*' and '?'
patterns.

If recursive is true, the pattern '**' will match any files and
zero or more directories and subdirectories.
File:      c:\users\zephy\anaconda3\envs\python_ml_guide\lib\glob.py
Type:      function


In [7]:
os.path.join(path, '*.data')

'D:\\workspace1\\github\\1_python_ml_guide\\UCI_opinion_review\\*.data'

In [8]:
glob.glob(os.path.join(path, '*.data'))

['D:\\workspace1\\github\\1_python_ml_guide\\UCI_opinion_review\\accuracy_garmin_nuvi_255W_gps.txt.data',
 'D:\\workspace1\\github\\1_python_ml_guide\\UCI_opinion_review\\bathroom_bestwestern_hotel_sfo.txt.data',
 'D:\\workspace1\\github\\1_python_ml_guide\\UCI_opinion_review\\battery-life_amazon_kindle.txt.data',
 'D:\\workspace1\\github\\1_python_ml_guide\\UCI_opinion_review\\battery-life_ipod_nano_8gb.txt.data',
 'D:\\workspace1\\github\\1_python_ml_guide\\UCI_opinion_review\\battery-life_netbook_1005ha.txt.data',
 'D:\\workspace1\\github\\1_python_ml_guide\\UCI_opinion_review\\buttons_amazon_kindle.txt.data',
 'D:\\workspace1\\github\\1_python_ml_guide\\UCI_opinion_review\\comfort_honda_accord_2008.txt.data',
 'D:\\workspace1\\github\\1_python_ml_guide\\UCI_opinion_review\\comfort_toyota_camry_2007.txt.data',
 'D:\\workspace1\\github\\1_python_ml_guide\\UCI_opinion_review\\directions_garmin_nuvi_255W_gps.txt.data',
 'D:\\workspace1\\github\\1_python_ml_guide\\UCI_opinion_review\\di

In [18]:
all_files = glob.glob(os.path.join(path, '*.data'))
filename_list = []
opinion_text = []

for file_ in all_files:
    # read each file and make DataFrame
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    # filename parsing
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]
    # filename list and file contents list
    filename_list.append(filename)
    opinion_text.append(df.to_string())
    
# filename list and contents list -> dataframe
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()    

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...
3,battery-life_ipod_nano_8gb,...
4,battery-life_netbook_1005ha,...


In [26]:
dict((ord(punct), None) for punct in string.punctuation)

{33: None,
 34: None,
 35: None,
 36: None,
 37: None,
 38: None,
 39: None,
 40: None,
 41: None,
 42: None,
 43: None,
 44: None,
 45: None,
 46: None,
 47: None,
 58: None,
 59: None,
 60: None,
 61: None,
 62: None,
 63: None,
 64: None,
 91: None,
 92: None,
 93: None,
 94: None,
 95: None,
 96: None,
 123: None,
 124: None,
 125: None,
 126: None}

In [38]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation) #punctuation unicode
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text): #make it lower case -> remove punctuation -> tokenize -> lemmatize
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))              

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', ngram_range=(1,2), min_df=0.05, max_df=0.85)

feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])
feature_vect.shape

(51, 4611)

In [43]:
from sklearn.cluster import KMeans
# clustering with n_clusters=5
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_
print(cluster_label)
print(cluster_centers)

[2 0 1 1 1 2 4 4 2 2 2 2 2 3 3 3 4 4 4 1 3 3 4 2 3 4 1 3 3 4 0 0 0 2 2 2 2
 4 3 3 3 1 1 2 1 3 3 4 2 2 2]
[[0.         0.         0.00474778 ... 0.         0.00159451 0.00067345]
 [0.01502839 0.         0.         ... 0.00105331 0.         0.        ]
 [0.00819396 0.         0.         ... 0.01050909 0.         0.        ]
 [0.         0.0012246  0.00068853 ... 0.         0.00176658 0.00157225]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]]


In [44]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,2
1,bathroom_bestwestern_hotel_sfo,...,0
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1


In [45]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,...,0
32,room_holiday_inn_london,...,0
30,rooms_bestwestern_hotel_sfo,...,0
31,rooms_swissotel_chicago,...,0


In [46]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1
19,keyboard_netbook_1005ha,...,1
26,performance_netbook_1005ha,...,1
41,size_asus_netbook_1005ha,...,1
42,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,1
44,speed_windows7,...,1


In [47]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,2
5,buttons_amazon_kindle,...,2
8,directions_garmin_nuvi_255W_gps,...,2
9,display_garmin_nuvi_255W_gps,...,2
10,eyesight-issues_amazon_kindle,...,2
11,features_windows7,...,2
12,fonts_amazon_kindle,...,2
23,navigation_amazon_kindle,...,2
33,satellite_garmin_nuvi_255W_gps,...,2
34,screen_garmin_nuvi_255W_gps,...,2


In [48]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
13,food_holiday_inn_london,...,3
14,food_swissotel_chicago,...,3
15,free_bestwestern_hotel_sfo,...,3
20,location_bestwestern_hotel_sfo,...,3
21,location_holiday_inn_london,...,3
24,parking_bestwestern_hotel_sfo,...,3
27,price_amazon_kindle,...,3
28,price_holiday_inn_london,...,3
38,service_bestwestern_hotel_sfo,...,3
39,service_holiday_inn_london,...,3


In [49]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
6,comfort_honda_accord_2008,...,4
7,comfort_toyota_camry_2007,...,4
16,gas_mileage_toyota_camry_2007,...,4
17,interior_honda_accord_2008,...,4
18,interior_toyota_camry_2007,...,4
22,mileage_honda_accord_2008,...,4
25,performance_honda_accord_2008,...,4
29,quality_toyota_camry_2007,...,4
37,seats_honda_accord_2008,...,4
47,transmission_toyota_camry_2007,...,4


In [65]:
# reduce number of clusters to 3
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
48,updates_garmin_nuvi_255W_gps,...,0
44,speed_windows7,...,0
43,speed_garmin_nuvi_255W_gps,...,0
42,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,0
41,size_asus_netbook_1005ha,...,0
36,screen_netbook_1005ha,...,0
35,screen_ipod_nano_8gb,...,0
34,screen_garmin_nuvi_255W_gps,...,0
33,satellite_garmin_nuvi_255W_gps,...,0


In [51]:
# Keyword extraction using cluster centers
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


In [66]:
km_cluster.cluster_centers_

array([[0.01005322, 0.        , 0.        , ..., 0.00706287, 0.        ,
        0.        ],
       [0.        , 0.00092551, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00099499, 0.00174637, ..., 0.        , 0.00183397,
        0.00144581]])

In [82]:
import numpy as np
np.array([[10,9,8], [1,2,3], [4,6,5]]).argsort()

array([[2, 1, 0],
       [0, 1, 2],
       [0, 2, 1]], dtype=int64)

In [83]:
np.array([[10,9,8], [1,2,3], [4,6,5]]).argsort()[:,::-1]

array([[0, 1, 2],
       [2, 1, 0],
       [1, 2, 0]], dtype=int64)

In [88]:
# returning top n keywords per cluster, relative position, filenames
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # descending order
    # distance 0~1 closer to 1 means closer to center
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    # iterate through clusters, and get keywords, relative position, filename
    for cluster_num in range(clusters_num):
        # dictionary initialization
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # get top n feature using indices obtained by cluster_centers_argsort()[:,::-1] 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [feature_names[ind] for ind in top_feature_indexes]
        
        # get relative position score
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # input values in the cluster_details dictionary
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label']==cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames']=filenames
        
    return cluster_details
    

In [92]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('##### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews filename:', cluster_detail['filenames'][:7])
        print('=================================================')

In [93]:
feature_names = tfidf_vect.get_feature_names()
cluster_details = get_cluster_details(cluster_model = km_cluster, cluster_data = document_df, 
                                      feature_names = feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

##### Cluster 0
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews filename: ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps']
##### Cluster 1
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews filename: ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyota_camry_2007', 'mileage_honda_accord_2008', 'performance_honda_accord_2008']
##### Cluster 2
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews filename: ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwes

In [100]:
a = WordNetLemmatizer() 
b = nltk.word_tokenize('I would like to be happier')
[a.lemmatize(x) for x in b]

['I', 'would', 'like', 'to', 'be', 'happier']